<a href="https://colab.research.google.com/github/Nevercomes/MDeepLearning/blob/ipynb/hello_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Built model

#### Import data from external sources
Import the terget image from google cloud storage:

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# from PIL import Image

# image = Image.open('/content/gdrive/My Drive/Colab Notebooks/pic2code/helloWorld_version/dataset/hw_img.png')
# image.show()

#### Import API

In [2]:
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

import numpy as np

print(tf.__version__)
print(tf.keras.__version__)
print(np.__version__)

1.12.0
2.1.6-tf
1.14.6


Using TensorFlow backend.


In [0]:
from keras.applications.vgg16 import VGG16
# from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [0]:
from keras.layers import Dense
from keras.layers import LSTM
# from keras.layers import RepeatVector

from keras.engine.input_layer import Input

In [0]:
from keras.backend import concatenate
from keras.models import Model

In [0]:
from keras.layers.core import RepeatVector

#### Preprocess the input image

In [9]:
# length of longest sentence
max_caption_len = 3
# size of vocabulary
vocab_size = 3

# load one screenshot for each word and turn them into digits
images = []
# the image comes from my google cloud storage
image = '/content/gdrive/My Drive/Colab Notebooks/pic2code/\
helloWorld_version/dataset/hw_img.png'
# image = Image.open(image)
image = load_img(image, target_size=(224,224))
w = 224
h = 224
c = 3
# image.resize((224,224))
# image.reshape(-1,w,h,c)


for i in range(2):
  image_array = img_to_array(image)
  print(image_array.shape)
  images.append(image_array)
images = np.array(images, dtype=float)
# preprocess input for the VGG16 model
images = preprocess_input(images)

(224, 224, 3)
(224, 224, 3)


** img_to_array: ** from *Keras.preprocessing.image*, converting images to arrays.
- target: or target_size, resize the image to target size.  

**VGG16 Model:**

#### Perprocess the tokens

In [0]:
# turn start tokens into one-hot encoding
html_input = np.array(
                [[[0., 0., 0.], #start
                 [0., 0., 0.],
                 [1., 0., 0.]],
                 [[0., 0., 0.], #start <HTML>Hello World!</HTML>
                 [1., 0., 0.],
                 [0., 1., 0.]]])
# turn next word into one-hot encoding
next_words = np.array(
                [[0., 1., 0.], # <HTML>Hello World!</HTML>
                 [0., 0., 1.]]) # end

#### Extract features by VGG16 trained on imageNet

In [11]:
# Load the VGG16 model trained on imagenet and output the classification feature
VGG = VGG16(weights='imagenet', include_top=True)
# Extract the features from the image
features = VGG.predict(images)

553467904/553467096 [==============================] - 5s 0us/step


#### Load the feature to the network

In [0]:
# Load the feature to the network, apply a dense layer, and repeat the vector
vgg_feature = Input(shape=(1000,))
vgg_feature_dense = Dense(5)(vgg_feature)
# keras.layers.RepeatVector(n) Repeats the input n times.
vgg_feature_repeat = RepeatVector(max_caption_len)(vgg_feature_dense)
# Extract information from the input seqence 
language_input = Input(shape=(vocab_size, vocab_size))
language_model = LSTM(5, return_sequences=True)(language_input)

在keras中，数据是以张量的形式表示的，张量的形状称之为shape，表示从最外层向量逐步到达最底层向量的降维解包过程。比如，一个一阶的张量[1,2,3]的shape是(3,);
一个二阶的张量[[1,2,3],[4,5,6]]的shape是(2,3);一个三阶的张量[[[1],[2],[3]],[[4],[5],[6]]]的shape是(2,3,1)。

input_shape就是指输入张量的shape。例如，input_dim=784，说明输入是一个784维的向量，这相当于一个一阶的张量，它的shape就是(784,)。因此，input_shape=(784,)。

input_dim = input_shape(input_dim,)

input_dim, input_length = input_shape(input_length, input_dim)

#### Concatenate, extract and predict

In [0]:
# Concatenate the information from the image and the input
decoder = concatenate([vgg_feature_repeat, language_model])
# Extract information from the concatenated output
decoder = LSTM(5, return_sequences=False)(decoder)
# Predict which word comes next
decoder_output = Dense(vocab_size, activation='softmax')(decoder)

# Compile and run the neural network
model = Model(inputs=[vgg_feature, language_input], outputs=decoder_output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

keras.backend.concatenate(tensors, axis=-1)  
Concatenates a list of tensors alongside the specified axis.  

**Arguments**

tensors: list of tensors to concatenate.
axis: concatenation axis.    

**Returns  **

A tensor.

#### Train

In [0]:
# Train the neural network
model.fit([features, html_input], next_words, batch_size=2, 
          shuffle=False, epochs=30)

In [14]:
    #Length of longest sentence
    max_caption_len = 3
    #Size of vocabulary 
    vocab_size = 3
    
    # Load one screenshot for each word and turn them into digits 
    images = []
    for i in range(2):
        images.append(img_to_array(load_img('/content/gdrive/My Drive/Colab Notebooks/pic2code/helloWorld_version/dataset/hw_img.png', target_size=(224, 224))))
    images = np.array(images, dtype=float)
    # Preprocess input for the VGG16 model
    images = preprocess_input(images)
    
    #Turn start tokens into one-hot encoding
    html_input = np.array(
                [[[0., 0., 0.], #start
                 [0., 0., 0.],
                 [1., 0., 0.]],
                 [[0., 0., 0.], #start <HTML>Hello World!</HTML>
                 [1., 0., 0.],
                 [0., 1., 0.]]])
    
    #Turn next word into one-hot encoding
    next_words = np.array(
                [[0., 1., 0.], # <HTML>Hello World!</HTML>
                 [0., 0., 1.]]) # end
    
    # Load the VGG16 model trained on imagenet and output the classification feature
    VGG = VGG16(weights='imagenet', include_top=True)
    # Extract the features from the image
    features = VGG.predict(images)
    
    #Load the feature to the network, apply a dense layer, and repeat the vector
    vgg_feature = Input(shape=(1000,))
    vgg_feature_dense = Dense(5)(vgg_feature)
    vgg_feature_repeat = RepeatVector(max_caption_len)(vgg_feature_dense)
    # Extract information from the input seqence 
    language_input = Input(shape=(vocab_size, vocab_size))
    language_model = LSTM(5, return_sequences=True)(language_input)
    
    # Concatenate the information from the image and the input
    decoder = concatenate([vgg_feature_repeat, language_model])
    # Extract information from the concatenated output
    decoder = LSTM(5, return_sequences=False)(decoder)
    # Predict which word comes next
    decoder_output = Dense(vocab_size, activation='softmax')(decoder)
    # Compile and run the neural network
    model = Model(inputs=[vgg_feature, language_input], outputs=decoder_output)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    # Train the neural network
    model.fit([features, html_input], next_words, batch_size=2, shuffle=False, epochs=1000)

AttributeError: ignored

### Make prediction

Get the output one by one, and splice for sentence to feed into the network manually.

In [0]:
# Create an empty sentence and insert the start token
sentence = np.zeros((1, 3, 3)) # [[0,0,0], [0,0,0], [0,0,0]]
start_token = [1., 0., 0.] # start
sentence[0][2] = start_token # place start in empty sentence
    
# Making the first prediction with the start token
second_word = model.predict([np.array([features[1]]), sentence])
    
# Put the second word in the sentence and make the final prediction
sentence[0][1] = start_token
sentence[0][2] = np.round(second_word)
third_word = model.predict([np.array([features[1]]), sentence])
    
# Place the start token and our two predictions in the sentence 
sentence[0][0] = start_token
sentence[0][1] = np.round(second_word)
sentence[0][2] = np.round(third_word)
    
# Transform our one-hot predictions into the final tokens
vocabulary = ["start", "<HTML><center><H1>Hello World!</H1></center></HTML>", "end"]
for i in sentence[0]:
    print(vocabulary[np.argmax(i)], end=' ')